In [1]:
import antlr4
from antlr4 import *
from BSLLexer import BSLLexer
from BSLParser import BSLParser
from BSLParserVisitor import BSLParserVisitor

input_stream = FileStream('ObjectModule.bsl', encoding='utf-8')
lexer = BSLLexer(input_stream)
stream = CommonTokenStream(lexer)
parser = BSLParser(stream)
tree = parser.file_()  # Пример именования стартового правила

In [2]:
from BSLMethodDescriptionLexer import BSLMethodDescriptionLexer
from BSLMethodDescriptionParser import BSLMethodDescriptionParser
from BSLMethodDescriptionParserVisitor import BSLMethodDescriptionParserVisitor
from antlr4 import InputStream

In [3]:
class MethodVisitor(BSLMethodDescriptionParserVisitor):
    def __init__(self):
        self.parse_dict={
            
        }

    def visitDeprecate(self, ctx):
        self.parse_dict.update({'deprecate': ctx.getText()})

    def visitDescriptionBlock(self, ctx):
        self.parse_dict.update({'descriptionBlock': ctx.getText()})
    
    def visitParameters(self, ctx):
        self.parse_dict.update({'parameters': ctx.getText()})
    
    def visitCallOptions(self, ctx):
        self.parse_dict.update({'callOptions': ctx.getText()})


In [4]:
def parse_comment(comment):
    string_stream = InputStream(comment)
    method_lexer = BSLMethodDescriptionLexer(string_stream)
    method_stream = CommonTokenStream(method_lexer)
    method_parser = BSLMethodDescriptionParser(method_stream)
    tree = method_parser.methodDescription()  # Пример именования стартового правила

    visitor = MethodVisitor()
    visitor.visit(tree)

    return visitor.parse_dict

In [5]:
"""parse_comment('//Hello, world \n// Параметры:\n // Устарела')"""

"parse_comment('//Hello, world \n// Параметры:\n // Устарела')"

In [6]:
def get_comments_above(ctx):
    met_non_empty_flag = False
    res_str = ""
    token_list = [item.text for item in [token for token in stream.getHiddenTokensToLeft(ctx.start.tokenIndex)]]
    for token in reversed(token_list):
        if '\n' in token:
            if met_non_empty_flag:
                break
        else:
            met_non_empty_flag = True
            res_str = ('\n'.join((res_str, token)) if res_str != '' else token)

    
    return res_str, parse_comment(res_str)

In [7]:

from BSLParser import BSLParser


class MyVisitor(BSLParserVisitor):
    def __init__(self, lexer):
        self.codeflow = list()
        self.childrenflow = list()
        self.lexer = lexer

    def visitTerminal(self, node):
        if node.symbol.type in [self.lexer.IF_KEYWORD, self.lexer.ELSE_KEYWORD, self.lexer.FOR_KEYWORD]: # предполагая, что IF, ELSE, FOR - константы токенов в вашем лексере
            print(f"KEYWORD'{node.getText()}") # модифицируем вывод
        else:
            print(node.getText())
    

In [8]:
keyws = set([
    "PROCEDURE",
    "FUNCTION",
    "ENDPROCEDURE",
    "ENDFUNCTION",
    "EXPORT",
    "VAL",
    "ENDIF",
    "ENDDO",
    "IF",
    "ELSIF",
    "ELSE",
    "THEN",
    "WHILE",
    "DO",
    "FOR",
    "TO",
    "EACH",
    "IN",
    "TRY",
    "EXCEPT",
    "ENDTRY",
    "RETURN",
    "CONTINUE",
    "RAISE",
    "VAR",
    "NOT",
    "OR",
    "AND",
    "NEW",
    "GOTO",
    "BREAK",
    "EXECUTE",
    "ADDHANDLER",
    "REMOVEHANDLER",
    "ASYNC"
])


def build_keyw_set(keyws, lexer, add_tokens=['LINE_COMMENT', ], add_preproc=False):
    keyws = set([
        getattr(lexer, kw + '_KEYWORD')
        for kw in keyws
    ] + [getattr(lexer, tok) for tok in add_tokens])
    return keyws

In [9]:
obj_keyws = build_keyw_set(keyws, lexer)

In [10]:
from antlr4.tree.Tree import TerminalNodeImpl

# Получение стартового токена для узла, относящегося к произвольному правилу
def getStartTok(node):
    """
    Получает индекс стартового токена для заданного узла дерева разбора,
    будь то контекстный узел или терминальный узел.
    
    Аргументы:
        node: Узел дерева разбора (RuleContext или TerminalNode).
        
    Возвращает:
        int: Индекс стартового токена данного узла.
    """
    if hasattr(node, 'symbol'):  # Для терминальных узлов
        return node.symbol.tokenIndex
    elif hasattr(node, 'start'):  # Для контекстных узлов
        return node.start.tokenIndex
    else:
        return None  # В случае, если у узла нет доступного стартового токена


In [159]:
import regex as re


def find_first_non_space(text):
    # Регулярное выражение для поиска пробельных символов и комментарив, чередующихся в любом порядке
    pattern = re.compile(r"(//.*\n|\s+)*")
    match = pattern.match(text)
    
    # Возвращаем индекс первого символа, не соответствующего шаблону
    if not match:
        return 0

    res = match.end()
    if res == len(text):
        return -1
    return res

In [160]:
find_first_non_space('   ')

-1

In [164]:
class UniversalVisitor(BSLParserVisitor):
    def __init__(self, stream, keyws=[]):
        self.codeflow = []
        self.defined_rule_mapping = {
            'ProcedureDeclarationCoreContext': 'PROC_DECL`',
            'FunctionDeclarationCoreContext': 'FUNC_DECL`',
            'ModuleVarDeclarationContext': 'VAR_NAME`',
            'SubVarDeclarationContext': 'VAR_NAME`',
            'DoCallContext': 'PARAM_LIST`',
            'AccessCallContext': 'ACCESS_CALL`',
            'MethodCall': 'METHOD_CALL`'
        }
        self.undefined_rule_mapping = {
            'SubNameContext': 'ID`',
            'ClosedParamListContext': 'ARGUMENTS`',
            'BeforeExecPartContext': 'ADDITIONAL_PART`'
        }
        self.undefined_prefix = {
            'FuncDeclarationContext': 'FUNC_',
            'ProcDeclarationContext': 'PROC_',
            'ProcedureDeclarationCoreContext': 'PROC_',
            'FunctionDeclarationCoreContext': 'FUNC_',
            '[default]': ''
        }
        self.keyws = keyws
        self.cur_father_node = '[default]'

        self.stream = stream

    def defaultResult(self):
        return ""

    def aggregateResult(self, aggregate, nextResult):
        #print(f'Aggregation: first = {aggregate}, second = {nextResult}')
        return aggregate + nextResult
    
    def visitTerminal(self, node):
        txt = node.getText()
        self.first_non_trash = 0
        if hasattr(node, 'symbol'):
            if (node.symbol.type in self.keyws):
                txt = 'KEYWORD`' + txt

            token_index = node.symbol.tokenIndex
            # Извлекаем токены из hidden channel, которые находятся слева от текущего токена
            hidden_tokens_to_left = self.stream.getHiddenTokensToLeft(token_index, antlr4.Lexer.HIDDEN) # 0 - это номер hiddenChannel
        
            # Добавляем текст каждого токена перед текущим токеном
            text = ''
            if hidden_tokens_to_left:
                for token in hidden_tokens_to_left:
                    text += token.text
            txt = text + txt

        return txt

    def visitChildren(self, node):
        # node – это текущий узел дерева
        name = node.__class__.__name__

        result = self.defaultResult()
        n = node.getChildCount()

        for i in range(n):
            c = node.getChild(i)
            # Рекурсивно посещаем всех детей узла
            prev_father = self.cur_father_node
            self.cur_father_node = name

            childResult = c.accept(self) #self.visitChildren(c, father_name=name)

            self.cur_father_node = prev_father
            result = self.aggregateResult(result, childResult)

        if name in self.defined_rule_mapping.keys():
            non_empty_idx = find_first_non_space(result)
            if non_empty_idx == -1:
                return result
            result = result[:non_empty_idx] + self.defined_rule_mapping[name] + result[non_empty_idx:]
        elif name in self.undefined_rule_mapping.keys():
            insertion = self.undefined_prefix[self.cur_father_node] + self.undefined_rule_mapping[name]
            non_empty_idx = find_first_non_space(result)
            if non_empty_idx == -1:
                return result
            result = result[:non_empty_idx] + insertion + result[non_empty_idx:]

        # Возвращаем ответ
        return result


In [165]:
vis = UniversalVisitor(
    stream,
    keyws=obj_keyws
)

In [166]:
out = vis.visit(tree)

In [167]:
print(out)

KEYWORD`Перем VAR_NAME`КэшПостроительДереваТестов;
KEYWORD`Перем VAR_NAME`ЗагружаемыйПуть;

KEYWORD`Перем VAR_NAME`КонтейнерТестов;
KEYWORD`Перем VAR_NAME`ТекущаяГруппа;

// { Plugin interface
FUNC_DECL`KEYWORD`Функция FUNC_ID`ОписаниеПлагинаFUNC_ARGUMENTS`(ВозможныеТипыПлагинов) KEYWORD`Экспорт
	Результат = KEYWORD`Новый Структура;
	РезультатACCESS_CALL`.ВставитьPARAM_LIST`("Тип", ВозможныеТипыПлагинов.Загрузчик);
	РезультатACCESS_CALL`.ВставитьPARAM_LIST`("Идентификатор", МетаданныеPARAM_LIST`().Имя);
	РезультатACCESS_CALL`.ВставитьPARAM_LIST`("Представление", "Загрузить тесты из файлов");
	// Test Comment
	KEYWORD`Возврат KEYWORD`Новый ФиксированнаяСтруктураPARAM_LIST`(Результат);
KEYWORD`КонецФункции

PROC_DECL`KEYWORD`Процедура PROC_ID`ИнициализацияPROC_ARGUMENTS`(КонтекстЯдраПараметр) KEYWORD`Экспорт
KEYWORD`КонецПроцедуры

// } Plugin interface

// { Loader interface
FUNC_ADDITIONAL_PART`#Если ТолстыйКлиентОбычноеПриложение Тогда
FUNC_DECL`KEYWORD`Функция FUNC_ID`ВыбратьПутьИнте

In [15]:
stream.channel

0